# Import libraries and declare functions

In [11]:
import numpy as np
import pickle
from PIL import Image
from skimage.feature import local_binary_pattern
import matplotlib.pyplot as plt
from curvelops import FDCT2D
from curvelops.plot import curveshow
from skimage import io, color
import pandas as pd
from scipy.signal import cwt, ricker
import multiprocessing
import cv2

convoluted_path = '/home/rajdeep/projects/image_segmentation/data/convoluted/'
path_to_groundtruth = './GroundTruth.csv'

# Function for extracting LBP from FDCT of images

In [12]:
def fdct_lbp(path):
    image_array = cv2.imread(path)
    #image = Image.open(path)
    #image_array = np.array(image)
    
    
    image_array = image_array.astype(float)
    image_array /= 255.0
    logo = image_array
    C2D  = FDCT2D(image_array.shape[:-1], nbscales=4, nbangles_coarse=16, allcurvelets=False)
    #coefficients = C2D.struct(C2D @ image_array)
    # Parameters for LBP
    subband = []
    logo_r = C2D.struct(C2D @ image_array[..., 0])
    logo_g = C2D.struct(C2D @ image_array[..., 1])
    logo_b = C2D.struct(C2D @ image_array[..., 2])

    # Concatenate the curvelet coefficients for each wedge
    logo_c = [[] for _ in logo_r]
    for iscale, c_angles in enumerate(logo_r):
        logo_c[iscale] = []
        for iwedge, c_wedge in enumerate(c_angles):
            wedges = [
                c[iscale][iwedge][..., np.newaxis].real
                for c in [logo_r, logo_g, logo_b]
            ]
            out = np.concatenate(wedges, axis=-1)
            # Rescaling because the curvelet coefficients are stronger than
            # the original signal due to the FFT normalization
            out *= np.sqrt(logo_r[iscale][iwedge].size / logo[..., 0].size)

            # However, the above scaling may put the signal outside of the [0, 1]
            # interval required for RGB images. As such we will rescale - purely
            # for visualization purposes, values between 0 and 1.
            out = (out - out.min()) / (out.max() - out.min())
            logo_c[iscale].append(out)
            subband.append(out)
    # Apply LBP to each curvelet coefficient
    radius = 1
    n_points = 8 * radius

    # Initialize a list to store LBP patterns
    lbp_patterns = []

    for coef in subband:
        #print(coef.shape)
        #image = color.rgb2gray(coef)
        lbp = local_binary_pattern(coef[0], n_points, radius, method='uniform')
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points + 3), range=(0, n_points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-7)
        lbp_patterns.append(hist)
    aggregated_features = np.concatenate(lbp_patterns)
    return aggregated_features

# Perfrom FDCT and extract LBP from images. 

In [19]:
df = pd.read_csv(path_to_groundtruth)


columns = ['MEL','NV','BCC','AKIEC','DF','VASC']
class_metadata = {} 
class_col = []
data_col = []
for col in columns:
    
    imgs = df[df.loc[:,col]==1]['image']
    imgs = map(lambda x:convoluted_path + x + '.jpg',imgs)
    category = col
    one_hot_encoded = df[df.loc[:,col]==1].loc[:,columns].values
    print(one_hot_encoded.shape)
    class_metadata[category] = one_hot_encoded[0] 
    with multiprocessing.Pool(processes=16) as pool:
        res = pool.map(fdct_lbp, imgs)
        data_col.append(res)
        class_col.append(one_hot_encoded)
        pool.close()
        pool.join()
  


(1, 1)


/home/rajdeep/projects/en/lib/python3.10/site-packages/skimage/feature/texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


# Save all the LBP features to a pickle file.

In [20]:
import pickle
data_card = {}
data_card['class_metadata'] = class_metadata
data_card['features'] = np.concatenate(data_col)
data_card['labels'] = np.concatenate(class_col)
print(data_card['features'].shape)
print(data_card['labels'].shape)
# File path to save the pickled dictionary
file_path = "./fdct_lbp.pkl"

# Save the dictionary as a pickled file
with open(file_path, "wb") as file:
    pickle.dump(data_card, file)

print("Dictionary saved as a pickled file.")


# Load the pickled dictionary
with open(file_path, "rb") as file:
    loaded_dict = pickle.load(file)
    print("Loaded dictionary:", loaded_dict.keys())

(1, 500)
(1, 1)
Dictionary saved as a pickled file.
Loaded dictionary: dict_keys(['class_metadata', 'features', 'labels'])
